# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd './drive/MyDrive/hasoc/task1'

/content/drive/MyDrive/hasoc/task1


# Installing Libraries

In [3]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install googletrans==4.0.0-rc1
# !pip install googletrans
!pip install accelerate -U
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 19.4 MB/s eta 0:00:

# Imports and Set Seed

In [4]:
import random
import torch
import numpy as np

GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


# Reading Translated Data

In [5]:
# unique_labels = df['label'].unique()
# label2id = {label: id for id, label in enumerate(unique_labels)}
# id2label = {id: label for id, label in enumerate(unique_labels)}
# num_labels = len(unique_labels)
unique_labels = ['HOF', 'NOT']
label2id = {'HOF': 0, 'NOT': 1}
id2label = {0: 'HOF', 1: 'NOT'}
num_labels = 2

In [6]:
id2label

{0: 'HOF', 1: 'NOT'}

# Defining Model and Tokenize Dataset

In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_checkpoint = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels, label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [9]:
# from datasets import load_dataset
# from transformers import DataCollatorForLanguageModeling
# from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# # dataset = load_dataset("your_dataset_name")  # Replace with your dataset name
# # dataset = ("your_dataset_name")
# from datasets import Dataset
# dataset = Dataset.from_pandas(df)
# # processed_dataset
# raw_datasets = dataset.train_test_split(test_size=0.1, seed=GLOBAL_SEED, shuffle=True)
# raw_datasets['validation'] = raw_datasets.pop('test')


# def tokenize_function(examples):
#     # return tokenizer(examples["text"], padding="max_length", truncation=True)
#     return tokenizer(examples["english_translation"], padding="max_length", truncation=True)

# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)#, remove_columns=['text', 'hate_label'])

In [10]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset, DatasetDict

# Load the dataset using the datasets library
# dataset = load_dataset("your_dataset_name")  # Replace with your dataset name

# Convert the dataset to a pandas DataFrame
# df = dataset.to_pandas()


# Split the dataset into train and test sets while maintaining class balance
# X = df.drop('label', axis=1)  # Replace 'target_column' with your actual target column name
# y = df['label']
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y, random_state=GLOBAL_SEED)

train_df = pd.read_csv('1B_train.csv', index_col = 0)
val_df = pd.read_csv('1B_val.csv', index_col = 0)

# Convert the training and testing sets back to datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenization
# tokenizer = AutoTokenizer.from_pretrained("your_pretrained_model")  # Replace with your tokenizer name
def tokenize_function(examples):
    return tokenizer(examples["english_translation"], padding="max_length", truncation=True)

tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_val_datasets = val_dataset.map(tokenize_function, batched=True)


tokenized_datasets = DatasetDict({"train": tokenized_train_datasets, "validation": tokenized_val_datasets})

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [11]:
# # Convert the train and validation datasets to pandas DataFrames
# train_df = tokenized_datasets["train"].to_pandas()
# val_df = tokenized_datasets["validation"].to_pandas()

# # Save train and validation DataFrames to CSV files
# train_df.to_csv("sinhala_train_dataset.csv", index=False)
# val_df.to_csv("sinhala_val_dataset.csv", index=False)

In [12]:
tokenized_datasets
# tokenized_train_datasets

DatasetDict({
    train: Dataset({
        features: ['created_at', 'text', 'user_screen_name', 'hate_label', 'english_translation', 'label', 'tweet_id', 'input_ids', 'attention_mask'],
        num_rows: 180
    })
    validation: Dataset({
        features: ['created_at', 'text', 'user_screen_name', 'hate_label', 'english_translation', 'label', 'tweet_id', 'input_ids', 'attention_mask'],
        num_rows: 20
    })
})

In [13]:
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer,
#     mlm=False  # Set to True if your task is Masked Language Modeling
# )
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Train Model

In [14]:
training_args = TrainingArguments(
    output_dir="./output",
    evaluation_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    learning_rate=2e-5,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=processed_dataset["train"],
    # eval_dataset=processed_dataset["validation"],
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator
)

# trainer.train(resume_from_checkpoint="last-checkpoint")
trainer.train()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.682360
2,No log,0.607335
3,No log,0.634560
4,No log,0.881958
5,No log,0.854537


TrainOutput(global_step=115, training_loss=0.49796772832455843, metrics={'train_runtime': 186.8108, 'train_samples_per_second': 4.818, 'train_steps_per_second': 0.616, 'total_flos': 32670993077280.0, 'train_loss': 0.49796772832455843, 'epoch': 5.0})

In [16]:
predictions = trainer.predict(tokenized_datasets["validation"])
# print(predictions.predictions.shape, predictions.label_ids.shape)
preds = np.argmax(predictions.predictions, axis=-1)

In [17]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Evaluate on validation data
results = trainer.evaluate()

# Calculate metrics
validation_loss = results["eval_loss"]
accuracy = accuracy_score(tokenized_datasets["validation"]["label"], preds)
precision, recall, f1, _ = precision_recall_fscore_support(tokenized_datasets["validation"]["label"], preds, average="weighted")

# Print the metrics
print(f"Validation Loss: {validation_loss}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Validation Loss: 0.6073354482650757
Accuracy: 0.6
Precision: 0.7
Recall: 0.6
F1 Score: 0.5878787878787879


In [18]:
test_df = pd.read_csv('gujarati_test_translated.csv', index_col = 0)


from datasets import load_dataset
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset, DatasetDict

# Convert the training and testing sets back to datasets
test_dataset = Dataset.from_pandas(test_df)


tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True)


tokenized_datasets["test"] = tokenized_test_datasets

Map:   0%|          | 0/1196 [00:00<?, ? examples/s]

In [19]:
# Save model predictions on the test dataset
test_predictions = trainer.predict(tokenized_datasets["test"])
test_preds = np.argmax(test_predictions.predictions, axis=-1)

# Save the predictions to a file (e.g., CSV)
# test_predictions_df = pd.DataFrame({"predictions": test_preds})
test_predictions_df = test_df.copy()
test_predictions_df['predictions'] = test_preds
test_predictions_df.to_csv("test_predictions_1B_xlmt.csv")

In [20]:
test_predictions_df

,text,english_translation,predictions
tweet_id,,,
2023_test_main_tweet_1,વધારે પડતાં લાગણીશીલ ન બનતા \nઆજના જમાના માં લ...,Don't get too emotional today an emotional per...,1
2023_test_main_tweet_2,@AUTHOR બસ કર બે ટોપા..,@AUTHOR Just do two topa..,0
2023_test_main_tweet_3,@AUTHOR @USER @USER બાયડી નો ભાઇ આયવો ભડવો 🤣😂,@AUTHOR @USER @USER Baidi's brother Aywo Bhadw...,0
2023_test_main_tweet_4,@AUTHOR ગુજરાત પોલીસ ઈસ તરહ બેવજહ મુસલમાન કો પ...,@AUTHOR Gujrat Police Is Tarah Bewajah Muslim ...,1
2023_test_main_tweet_5,@AUTHOR ગુજરાત માં ભરૂચ જિલ્લામાં આવેલો જંબુસર...,@AUTHOR Municipal employees in Jambusar taluka...,0
...,...,...,...
2023_test_main_tweet_1192,"@AUTHOR સાચું કહે છે, પણ નકલી ગાંધી છે\n(૧) નક...","@AUTHOR is right, but fake Gandhi is.(1) Fake ...",1
2023_test_main_tweet_1193,@AUTHOR ચોર છે બધા,@AUTHOR Thieves are all,0
2023_test_main_tweet_1194,@AUTHOR મંદ ચાલશે પણ ચોર નઈ ચાલે,@AUTHOR Slow will run but Thief won't,1
